# Environment II (0.8-quantile)

In this notebook, we show the simulation for the Group Two design.

We consider $K = 20 + m$ arms with 15 arm A, 5 arm B and $m$ arm C (optimal arms).
The goal is to identify $m$ arms with maximum $0.8$-quantile.
Both mean and 0.8-quantile provide the same order of arms, while
0.8-quantile can provide a larger gap compared with the mean.
The environment with larger minimum suboptimality gap has smaller probability complexity
and thus smaller upper bound of probability of error (it holds for the mean-based algorithm \citep{bubeck2013multiple} as well).
We compare our algorithms with the baseline algorithms:
- mean-based Successive Accepts and Rejects (SAR) \citep{bubeck2013multiple}.
- mean-based Successive Rejects (SR) \citep{audibert2010best}.
- Quantile Successive Rejects (Q-SR). 

In [7]:
# direct to proper path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
import pickle
from prettytable import PrettyTable

from codes.BAI_FixedBudget import *
from codes.Environment import Mixture_AbsGau, setup_env, Exp
from codes.SimulatedGames import simulate
from codes.plots import *

%matplotlib inline

In [8]:

# setting
results = defaultdict(dict)
num_exper = 5000
epsilon = 0


m = 5 # number of recommendation
tau = 0.8 # quantile level

if m == 1:
  budget_list = [500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500]
elif m == 5:
  budget_list = [1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 3750, 4000]

# output/plot setting

output_file = '../Group_Two_'  + 'm' + str(m) + '_exper_' + str(num_exper) + '_' + 'budget_' + str(budget_list[0]) + '_' + str(budget_list[-1]) 
budget_plot_title = 'BAI with '+ str(tau) + ' quantile (m=' + str(m) + ')'

In [9]:
# environment

environment_m1 = [
    {Mixture_AbsGau: [[0, 2, 0,1, 1], [3.5, 2, 0, 1, 1], 
                      [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], 
                      [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1],  
                     ],
     Exp: [[1/4], # [1/4], [1/4], [1/4], [1/4]
     ]
    } # mu1, sigma1, mu2, sigma2, p 
]

environment_m5 = [
    {Mixture_AbsGau: [[0, 2, 0,1, 1], [3.5, 2, 0, 1, 1], 
                      [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], 
                      [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1], 
                      [0, 2, 0,1, 1],  
                      [0, 2, 0,1, 1],  
                     ],
     Exp: [[1/4], [1/4], [1/4], [1/4], [1/4]
     ]
    } # mu1, sigma1, mu2, sigma2, p 
]

if m == 1:
  environments = environment_m1
elif m == 5:
  environments = environment_m5
else: 
  print('Unknown environment!')

rewards_env_quant, true_ss_dict_quant, samples_quant = setup_env(environments, ['quantile_' + str(tau)])
rewards_env_mean, true_ss_dict_mean, samples_mean = setup_env(environments, ['mean'])

{<class 'codes.Environment.Mixture_AbsGau'>: [[0, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1]], <class 'codes.Environment.Exp'>: [[0.25], [0.25], [0.25], [0.25], [0.25]]}
{<class 'codes.Environment.Mixture_AbsGau'>: [[0, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [3.5, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1], [0, 2, 0, 1, 1]], <class 'codes.Environment.Exp'>: [[0.25], [0.25], [0.25], [0.25], [0.25]]}


In [10]:
# run experiments

def run_exper(rewards_env, summary_stats, policy, epsilon, m, budget, num_exper, fixed_samples_list = None,policy_for_mean = False):
    for key in rewards_env.keys():
        name = key + '_' + str(num_exper)
        if policy_for_mean:
                policy_name = str(policy).split('.')[-1].split('\'')[0] + '-'
                policy_name = policy_name[2:]
                # print(policy_name)
        else:
                policy_name = str(policy).split('.')[-1].split('\'')[0] + '-'

        subname = policy_name  + str(budget) 
        p = IntProgress(max = num_exper)
        p.description = 'Running'
        display(p)
        results[name][subname] = []

        result = simulate(rewards_env[key],summary_stats[key], policy, 
                epsilon, m, budget, num_exper, p, fixed_samples_list)
        results[name][subname].append(result)

In [11]:

for budget in budget_list:
    run_exper(rewards_env_quant, true_ss_dict_quant, Q_SR, epsilon, m, budget, num_exper)
    run_exper(rewards_env_quant, true_ss_dict_quant, Q_SAR_Simplified, epsilon, m, budget, num_exper)
    run_exper(rewards_env_mean, true_ss_dict_mean, Q_SR, epsilon, m, budget, num_exper, policy_for_mean = True)
    run_exper(rewards_env_mean, true_ss_dict_mean, Q_SAR_Simplified, epsilon, m, budget, num_exper, policy_for_mean = True)

IntProgress(value=0, description='Running', max=5000)

IntProgress(value=0, description='Running', max=5000)

IntProgress(value=0, description='Running', max=5000)

IntProgress(value=0, description='Running', max=5000)

IntProgress(value=0, description='Running', max=5000)

...

IntProgress(value=0, description='Running', max=5000)

IntProgress(value=0, description='Running', max=5000)

IntProgress(value=0, description='Running', max=5000)

IntProgress(value=0, description='Running', max=5000)

IntProgress(value=0, description='Running', max=5000)

In [12]:
# save results

with open(output_file, 'wb') as handle:
     pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

plot_eva_budget(results, 'pe', paper_flag = True, with_para = True, title = budget_plot_title, log_scale = False)

Budget  Probability of Error       std Policy
0   1000                0.7418  0.043764   Q-SR
1   1000                0.5816  0.049330  Q-SAR
2   1000                0.9122  0.028300     SR
3   1000                0.8602  0.034678    SAR
4   1250                0.6556  0.047517   Q-SR


Budget  Probability of Error       std Policy
0    1000                0.7418  0.043764   Q-SR
1    1000                0.5816  0.049330  Q-SAR
2    1000                0.9122  0.028300     SR
3    1000                0.8602  0.034678    SAR
4    1250                0.6556  0.047517   Q-SR
5    1250                0.4810  0.049964  Q-SAR
6    1250                0.8842  0.031998     SR
7    1250                0.8166  0.038699    SAR
8    1500                0.5678  0.049538   Q-SR
9    1500                0.4052  0.049093  Q-SAR
10   1500                0.8584  0.034864     SR
11   1500                0.7676  0.042236    SAR
12   1750                0.4716  0.049919   Q-SR
13   1750                0.3160  0.046491  Q-SAR
14   1750                0.8266  0.037859     SR
15   1750                0.7332  0.044229    SAR
16   2000                0.4126  0.049230   Q-SR
17   2000                0.2594  0.043831  Q-SAR
18   2000                0.7774  0.041599     SR
19   2000                0.6784  0.046709    SAR
20   2250                0.3446  0.047524   Q-SR
21   2250                0.2014  0.040105  Q-SAR
22   2250                0.7456  0.043552     SR
23   2250                0.6180  0.048588    SAR
24   2500                0.2916  0.045450   Q-SR
25   2500                0.1578  0.036455  Q-SAR
26   2500                0.7008  0.045791     SR
27   2500                0.5774  0.049397    SAR
28   2750                0.2454  0.043032   Q-SR
29   2750                0.1296  0.033586  Q-SAR
30   2750                0.6726  0.046926     SR
31   2750                0.5454  0.049793    SAR
32   3000                0.2068  0.040501   Q-SR
33   3000                0.1056  0.030732  Q-SAR
34   3000                0.6458  0.047827     SR
35   3000                0.4996  0.050000    SAR
36   3250                0.1772  0.038184   Q-SR
37   3250                0.0804  0.027191  Q-SAR
38   3250                0.5984  0.049022     SR
39   3250                0.4442  0.049688    SAR
40   3500                0.1372  0.034406   Q-SR
41   3500                0.0636  0.024404  Q-SAR
42   3500                0.5676  0.049541     SR
43   3500                0.4072  0.049131    SAR
44   3750                0.1142  0.031805   Q-SR
45   3750                0.0456  0.020862  Q-SAR
46   3750                0.5330  0.049891     SR
47   3750                0.3812  0.048568    SAR
48   4000                0.1026  0.030344   Q-SR
49   4000                0.0420  0.020059  Q-SAR
50   4000                0.4900  0.049990     SR
51   4000                0.3464  0.047582    SAR